In [2]:
import numpy as np 
import random
import math
import pandas as pd
import matplotlib.pyplot as plt
import skfda
from functions import *
from sklearn.model_selection import train_test_split
from skfda.ml.regression import LinearRegression
from skfda.representation.basis import BSplineBasis
from skfda.ml.regression import KNeighborsRegressor
from skfda.misc.kernels import uniform
from skfda.preprocessing.smoothing import KernelSmoother
from skfda.representation.basis import FDataBasis
from skfda.exploratory.visualization import Boxplot
from skfda.preprocessing.missing import MissingValuesInterpolation
from collections import Counter
from matplotlib.colors import LinearSegmentedColormap
import matplotlib
from matplotlib import cm
import seaborn as sns


## Data preprocessing - Output processing

In [ ]:
# Read data
mto_df = pd.read_excel('...')
mto_df = mto_df.iloc[:,4:]

# Set column names
col_xy = list(mto_df.columns)
col_x = list(mto_df.columns[0:15])

# Define target yield variable
Yield = 'Sethylene%'
mto_df['Yield_1'] = mto_df['Sethylene%']/100
mto_df['Yield'] = mto_df['Sethylene%']/100
mto_df['ID'] = None

# Initialize ID counter
k = 0

# Iterate through each row to assign IDs
for i, row in mto_df.iterrows():
    if pd.notna(mto_df.at[i, 'ID']):
        continue
    mto_df.at[i, 'ID'] = k
    in_row_x = row[col_x]
    for j in range(i+1, len(mto_df)):
        if (in_row_x == mto_df.loc[j, col_x]).all():
            if mto_df.at[j, 'TOS(min)'] > mto_df.at[j-1, 'TOS(min)']:
                mto_df.at[j, 'ID'] = k
            else:
                break
        else:
            break
    k += 1

print(mto_df)

# Group by ID and aggregate information
Info = (
    mto_df
    .groupby('ID')
    .agg(
        n=('TOS(min)', lambda x: sum(x.notna() & mto_df['Yield_1'].notna())),
        maxTOS=('TOS(min)', 'max'),
        minTOS=('TOS(min)', 'min'),
    )
    .reset_index()
)

print(Info.head())

# Process maximum TOS for each ID
grouped = mto_df.loc[:,['ZEOTYPE','TOS(min)','ID']].groupby('ID')
max_tos = [grp['TOS(min)'].max() for i,grp in grouped]
max_tos_ID = [grp.iloc[0,2] for i,grp in grouped]
max_tos_z = [grp.iloc[0,0] for i,grp in grouped]

# Create DataFrame
max_zt_df = pd.DataFrame()
max_zt_df['ID']=max_tos_ID
max_zt_df['ZEOTYPE']=max_tos_z
max_zt_df['max_tos']=max_tos

In [ ]:
# Analyze the maximum experimental time
max_tos_Q1 = np.percentile(max_tos, 25)
max_tos_Q2 = np.percentile(max_tos, 75)
print('The lower quartile of the maximum time list:', max_tos_Q1)
print('The upper quartile of the maximum time list:', max_tos_Q2)

below_Q1 = [x for x in max_tos if x <= max_tos_Q1]
normal = [x for x in max_tos if (x > max_tos_Q1 and x <= max_tos_Q2)]
out_Q1 = [x for x in max_tos if x > max_tos_Q1]
print('Number of curves:', len(max_tos))
print('Curves below the lower quartile:', len(below_Q1))
print('Curves in the interquartile range:', len(normal))
print('Curves above the upper quartile:', len(out_Q1))

# Check the number of zeotype types in each segment
m1 = np.unique(max_zt_df.ZEOTYPE[max_zt_df['max_tos'] <= max_tos_Q1])
m2 = np.unique(max_zt_df.ZEOTYPE[max_zt_df['max_tos'] >= max_tos_Q1])
print('Zeotype types below the lower quartile:', m1)
print('Zeotype types above the upper quartile:', m2)

# Plot the scatter plot of curves below the lower quartile
plt.scatter(below_Q1, [1] * len(below_Q1), s=10, marker='.')
plt.title('Below Q1 Scatter plot')
plt.show()

# Plot the scatter plot of curves in the interquartile range
plt.scatter(normal, [1] * len(normal), s=10, marker='.', c='red')
plt.title('Normal Scatter plot')
plt.show()

# Plot the scatter plot of curves above the upper quartile
plt.scatter(out_Q1, [1] * len(out_Q1), s=10, marker='.', c='red')
plt.title('Above Q1 Scatter plot')
plt.show()

'''First perform modeling analysis on curves below the lower quartile.'''
'''For curves above the upper quartile, further binning can be considered.'''

In [ ]:
# IDs that meet the research criteria
# use_ID = Info.loc[(max_tos<=max_tos_Q2) & (Info['n'] >= 3) & (max_tos>=max_tos_Q1), 'ID'].tolist()
use_ID = Info.loc[(Info['maxTOS'] <= 3000) & (Info['n'] >= 5) & (Info['maxTOS'] >= 300),'ID'].tolist()
use_mto_df = mto_df[mto_df['ID'].isin(use_ID)]
use_mto_df.reset_index(drop=True,inplace=True)
use_mto_df.to_excel('use_df.xlsx', index=False)

# Define the time interval and interpolation points
# - the time interval we focused on
Tmin = 0; Tmax = 300
Grids = np.arange(Tmin, Tmax+1, 1)

'''Call the defined function for interpolation'''
df_Curves = format_Curves(use_mto_df, use_ID, 'Stotal%', Grids)
print(len(df_Curves))

In [ ]:
curves_show_id = [0, 28, 93, 10, 31, 87]

plt.rc('font', size=16)
fig, axs = plt.subplots(1, 2, figsize=(14, 6), dpi=300)
f_data = df_Curves
# f_data = data_y_kns
axs = axs.flatten()
j=0
for i in range(2):
    ax=axs[i]
    curves = f_data.iloc[curves_show_id[3*i], :].tolist()
    curves_1 = f_data.iloc[curves_show_id[3*i+1], :].tolist()
    curves_2 = f_data.iloc[curves_show_id[3*i+2], :].tolist()
    ax.plot(range(len(curves)), curves, linewidth=2, ls='-', label=f'Curve {curves_show_id[3*i]+1}')
    ax.plot(range(len(curves)), curves_1, linewidth=2, ls='-.', label=f'Curve {curves_show_id[3*i+1]+1}')
    ax.plot(range(len(curves)), curves_2, linewidth=2, ls='--', label=f'Curve {curves_show_id[3*i+2]+1}')
    if i==0:
        ax.set_ylim(0.15,0.62)
    else:
        ax.set_ylim(0,0.3)
    ax.legend(fontsize=15)
    ax.set_xlim(5, 300)
    ax.set_xticks([5, 100, 200, 300])
    ax.set_xlabel('TOS (min)', fontsize=21)
    ax.set_ylabel('Ethylene selectivity', fontsize=21)
    ax.tick_params(axis='both', which='major', labelsize=19)
    ax.legend(fontsize=15,loc='upper right')
plt.tight_layout()
plt.savefig(f"...")
plt.show()

## Outlier handling

In [ ]:
df_Curves_copy = df_Curves.copy()
# Calculate the first-order derivative
df_diff = df_Curves_copy.diff(axis=1)
# Set the threshold
threshold = 2 * df_diff.std().mean()  # For example, use twice the mean standard deviation as the threshold

# Identify outliers
outliers = np.abs(df_diff) > threshold

# Mark curves containing outlier values
outlier_indices = outliers.any(axis=1)
print(f"Identified {len(outlier_indices[outlier_indices])} curves with outlier points")
# Replace outlier points with NaN
df_Curves_copy[outliers] = np.nan
# Calculate the number of curves containing outlier values
num_outlier_curves = sum(outlier_indices)

In [12]:
df_Curves = df_Curves_copy

In [ ]:
## Format adjustment
# Set the time steps
data_tpts = Grids
# Set the response variable y
data_y = df_Curves.reset_index(drop=True)
data_y[data_y < 0] = np.nan

In [14]:
## Kernel Smoother
data_y_f = skfda.FDataGrid(
    data_matrix=data_y,
    grid_points=data_tpts)
nan_interp = MissingValuesInterpolation()
data_y_f = nan_interp.fit_transform(data_y_f)

fd_os = KernelSmoother(
    kernel_estimator=NadarayaWatsonHatMatrix(bandwidth=6, kernel=uniform),
).fit_transform(data_y_f)
data_y_kns = pd.DataFrame(fd_os.data_matrix.reshape(-1,len((fd_os.data_matrix[0]))),columns=data_tpts)

In [17]:
data_y = data_y_kns

## Data preprocessing - Input variable processing

In [ ]:
X_name = ['Modification','AS','A/T','FDSi','Largest Ring sizes','MDa',
          'MDb','MDc','Mdi','CD','crystal size(μm)','reaction temp(°C)',
          'WHSV(h-1)']
cla_name = ['Modification', 'AS', 'Largest Ring sizes', 'CD']
num_name = [n for n in X_name if n not in cla_name]

X_df = use_mto_df.drop_duplicates(subset='ID', keep='first')
data_x = X_df[X_name].reset_index(drop=True)
data_x = data_x.loc[index_to_keep]
data_x = data_x.reset_index(drop=True)

In [ ]:
X_name = ['ZEOTYPE','Modification','AS','A/T','FDSi','Largest Ring sizes','MDa',
          'MDb','MDc','Mdi','CD','crystal size(μm)','reaction temp(°C)',
          'WHSV(h-1)']
X_df = use_mto_df.drop_duplicates(subset='ID', keep='first')
data_x = X_df[X_name].reset_index(drop=True)
data_x_ac = data_x.loc[index_to_keep]
data_x_ac = data_x_ac.reset_index(drop=True)

In [ ]:
## Zeolite Type Chart
zeotype_data = data_x_ac['ZEOTYPE'].tolist()
# Calculate the frequency of each category
counts = Counter(zeotype_data)

# Get the category labels and their corresponding frequency values
labels, values = zip(*counts.items())

# Sort the labels and their values by frequency in descending order
sorted_labels_values = sorted(zip(labels, values), key=lambda x: x[1], reverse=True)
labels_sorted, values_sorted = zip(*sorted_labels_values)

# Create a gradient color map
cmap = LinearSegmentedColormap.from_list("mycmap", [(0, "#9EC6DB"),(0.5,"#C5E3E2"), (1, "#EDF4F5")])

matplotlib.rcParams['font.size'] = 12
# Create a new figure
fig, ax = plt.subplots(dpi=300)

# Plot the bar chart using the color map
bars = ax.bar(labels_sorted, values_sorted, color=cmap(np.linspace(0, 1, len(values_sorted))))

# Optional: Add a title and axis labels to the chart
ax.set_xlabel('Zeolite Types', fontsize=13)
ax.set_ylabel('Frequency', fontsize=13)

# Rotate the x-axis labels by 45 degrees
plt.xticks(rotation=45, ha='right')

# Display the chart
plt.tight_layout()  # Automatically adjust subplot parameters to fill the entire image area
# Save the chart as a PNG file
plt.savefig('...')
plt.show()

In [ ]:
# Use the 'Blues' colormap, which is a gradient from light blue to dark blue
cmap = cm.PuBu
# cmap = cm.Wistia

fig = plt.figure(figsize=(10, 8), dpi=300)
# Create a Boxplot object
boxplot = Boxplot(fd_os, depth_method=IntegratedDepth(), prob=[0.75, 0.5, 0.25], fig=fig)
boxplot.colormap = cmap
boxplot.plot()
# Since Boxplot automatically creates axis objects, we need to retrieve them from the fig object
ax = fig.axes[0]  # Get the first (and only) axis object
# Set the x-axis and y-axis labels
ax.set_xticks([5, 100, 200, 300])
ax.set_xlabel('TOS (min)', fontsize=18)  # Set the x-axis label and font size
ax.set_ylabel('Ethylene selectivity', fontsize=18)  # Set the y-axis label and font size
ax.tick_params(axis='both', which='major', labelsize=16)  # Adjust the tick label font size
plt.savefig('...')
# Display the plot
plt.show()

In [ ]:
row_means = data_y.apply(np.mean, axis=1)
data_x_ac['se_mean'] = row_means

In [ ]:
data_x_ac.to_csv('...', index=False)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 10), dpi=600)
axs = axs.flatten()
columns = ['Modification', 'AS', 'Largest Ring sizes', 'CD']
new_x_labels = ['Mod', 'AS', 'LRS', 'CD']
palette = 'pastel'
for i, col in enumerate(columns):
    sns.violinplot(x=col, y='se_mean', data=data_x_ac, ax=axs[i], palette=palette)
    axs[i].set_xlabel(new_x_labels[i], fontsize=17)
    axs[i].set_ylabel('Mean value of ethylene selectivity', fontsize=17)
    axs[i].set_xticks(axs[i].get_xticks())
    axs[i].set_xticklabels(axs[i].get_xticklabels(), fontsize=16)
    axs[i].tick_params(axis='y', labelsize=14)

plt.subplots_adjust(hspace=0.26,wspace=0.26)
plt.savefig('...', format='SVG')
plt.show()